In [1]:
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime


In [2]:
def get_link(dir, start_date, end_date=False, Moscow_arpts=True, airport = ''):
    '''
    Функция возвращает список сслылок с сайта http://mvsm.ru/Tablo.aspx для последующего парсинга, принимая на вход 
    следующие аргументы:
    - 'dir' - направление рейса. 0 - вылет, 1 - прилет
    - start_date - начальная дата сбора информации (формат - YYYY-mm-dd), самое раннее значение - 2021-12-31.
    - end_date - дата завершения сбора информации (формат - YYYY-mm-dd). По умолчанию - текущие дата и время.
    - Moscow_arpts - по умолчанию - True, собирает информацию по аэропортам Домодедово, Шереметьево, Внуково. 
      При параметре False требуется указание параметра airports
    - airport - принимает на вход одно значение: 'DME'(Домодедово), 'SVO'(Шереметьево) или 'VKO'(Внуково).
    '''
    
    URL = 'http://mvsm.ru/Tablo.aspx'
    links_list = list()
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if end_date:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    else:
        end_date = datetime.now()
        
    while start_date <= end_date:
        link = str(URL + (f'?Dir={dir}&Date={start_date.strftime("%Y-%m-%d")}&Hour={start_date.hour}'))
        if Moscow_arpts:
            link += (f'&City=Москва')
        if airport != '':
            link += (f'&Airport={airport}')
        links_list.append(link)
        
        start_date += timedelta(hours=8)
    
    return links_list 

# пример вывода URL-адреса с параметрами: http://mvsm.ru/Tablo.aspx?Dir=1&Date=2023-02-08&Hour=23&City=Москва&Airport=DME
    

In [3]:
res_links = get_link(0, '2022-11-26', '2023-03-01')

In [4]:
def get_flight_info_res(links_list):
    
    res_flights_info = pd.DataFrame()
    try:
        for link in links_list:
            try:
                response = requests.get(link)
                time.sleep(2)
                soup = BeautifulSoup(response.text, 'html5lib')
                flights_info = soup.find_all('tr')           

                for el in list(flights_info)[30:]:
                    if el.find('a'):

                        fl_id = el.find('a', alt="Подписка").get('href')
                        fl_id = str(fl_id)[-8:]
                        if fl_id not in res_flights_info.values:                            

                            company_name = el.find('td', align='center')['title']
                            
                            company_code_pattern = str(re.search(r'\"Aircompany.aspx\?Code=\w+"', str(el)).group()).replace('"', '')
                            company_code = el.find('a', href=company_code_pattern).text
       
                            flight_number_pattern = str(re.search(r'\"FlightOnDate.aspx\?.*"', str(el)).group()).replace('"', '')
                            fl_string = str(flight_number_pattern[-26:-19])
                            flight_number = re.search(r'\s[0-9]{2,5}', fl_string).group().strip()

                            completed_flight = True
                            times_list = list()
                            time_pattern = '\d{2}.\d{2}.\d{4}\s\d{1,2}:\d{2}:\d{2}'
                            for i in el.find_all('td', align="center"):
                                if re.findall(time_pattern, i.text):
                                    times_list.append(str(re.search(time_pattern, i.text).group()))
                                if i.get_text(strip=True) == 'Отменен':
                                    completed_flight = False
                                    break
                                pass
                            plan_time, fact_time = times_list

                            aircraft = None
                            aircraft_pattern = '[АБВЕИЛМСABCDEFJLM]{1}\d\w{1,2}|32[01]{1}|Boeing\s\w+|Sukhoi\s\w+\s\w+|SSJ\d{2,3}'
                            for j in el.find_all('td', align="center"):
                                if re.search(aircraft_pattern, j.text):
                                    aircraft = (re.search(aircraft_pattern, j.text).group().replace('Б', 'B')
                                                .replace('В', 'B').replace('Е', 'E').replace('oeing', '').replace('320', 'A 320')
                                                .replace('AA', 'A').replace('А', 'A').replace('irbus', '')
                                                .replace('M83', 'MD83').replace('Sukhoi Superjet', 'SSJ')
                                                .replace(' ', '').replace('21N', '321').replace('20N', '320')
                                                .replace('B77W', 'B777').replace('J200', 'CRJ200').replace('E75L', 'E175')
                                                .replace('b38m', 'B738').replace('L76', 'Il76').replace('B39M', 'B739')
                                               )
                                    break
                                pass              
                                                      
                            airports_list = list()
                            for i in el.find_all('td', align="left"):
                                airports_list.append(str(i.get_text(strip=True)))       
                                
                            dep_airport = airports_list[0].upper()
                            arr_airport = airports_list[2].upper()
                            
                            row = {'Flight_id': fl_id, 'Company_name': company_name, 'Company_code': company_code,
                                   'Flight_number': flight_number, 'Plan_time': plan_time, 'Fact_time': fact_time,
                                    'Completed_flight': completed_flight, 'Aircraft': aircraft, 
                                    'Departure_airport': dep_airport, 'Arrival_airport': arr_airport}
                                
                            res_flights_info = pd.concat([res_flights_info, pd.DataFrame([row])])

            except Exception as e:
                print(link[37:47], end = ', ')
                pass
                            
    finally:
        return res_flights_info

In [5]:
result = get_flight_info_res(res_links)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_16360\2433006324.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if fl_id not in res_flights_info.values:


2022-11-26, 2022-11-26, 2022-11-26, 2022-11-27, 2022-11-27, 2022-11-27, 2022-11-28, 2022-11-28, 2022-11-28, 2022-11-29, 2022-11-29, 2022-11-29, 2022-11-30, 2022-11-30, 2022-11-30, 2022-12-01, 2022-12-01, 2022-12-01, 2022-12-02, 2022-12-02, 2022-12-02, 2022-12-03, 2022-12-03, 2022-12-03, 2022-12-04, 2022-12-04, 2022-12-04, 2022-12-05, 2022-12-05, 2022-12-05, 2022-12-06, 2022-12-06, 2022-12-06, 2022-12-07, 2022-12-07, 2022-12-07, 2022-12-08, 2022-12-08, 2022-12-08, 2022-12-09, 2022-12-09, 2022-12-09, 2022-12-10, 2022-12-10, 2022-12-10, 2022-12-11, 2022-12-11, 2022-12-11, 2022-12-12, 2022-12-12, 2022-12-12, 2022-12-13, 2022-12-13, 2022-12-13, 2022-12-14, 2022-12-14, 2022-12-14, 2022-12-15, 2022-12-15, 2022-12-15, 2022-12-16, 2022-12-16, 2022-12-16, 2022-12-17, 2022-12-17, 2022-12-17, 2022-12-18, 2022-12-18, 2022-12-18, 2022-12-19, 2022-12-19, 2022-12-19, 2022-12-20, 2022-12-20, 2022-12-20, 2022-12-21, 2022-12-21, 2022-12-21, 2022-12-22, 2022-12-22, 2022-12-22, 2022-12-23, 2022-12-23, 2022

In [6]:
result.shape

(59372, 10)

In [7]:
rus_companies_list = ['Аэрофлот', 'Россия', 'Победа', 'S7', 'Уральские', 'Ural Airlines', 'Северный', 'Nordwind',
                      'Ямал', 'Pegas Fly', 'IrAero', 'Ираэро', 'Ред Вингс', 'Red Wings', 'Utair', 'Azur Air', 'Royal Flight', 
                      'Smartavia', 'Нордавиа', 'NordStar', 'Северсталь', 'Руслайн', 'Алроса', 'Ижавиа', 'Азимут', 'Якутия', 
                      'SmartWings', 'Газпром', 'ЮВТ', 'ВОЛОГОДСКОЕ', 'Северо-Запад', 'I Fly', 'IFly', 'РусДжет', 'Таймыр'
                     ]

def search_rus_aircompanies(company_name):
    for i in rus_companies_list:
        if i in company_name:
            return 1    

In [8]:
result['rus_company'] = result['Company_name'].apply(search_rus_aircompanies)
result.head(5)

,Flight_id,Company_name,Company_code,Flight_number,Plan_time,Fact_time,Completed_flight,Aircraft,Departure_airport,Arrival_airport,rus_company
0,18581429,SCAT,DV,810,25.11.2022 19:15:00,25.11.2022 23:35:00,True,None,МОСКВА (ШЕРЕМЕТЬЕВО),АЛМАТЫ (АЛМАТЫ),NaN
0,18588380,Utair,UT,785,25.11.2022 19:35:00,25.11.2022 21:02:00,True,B737,МОСКВА (ВНУКОВО),ЕРЕВАН (ЗВАРТНОЦ ИНТЕРН.),1.0
0,18589617,S7 Airlines,S7,1019,25.11.2022 20:15:00,25.11.2022 21:04:00,True,A320,МОСКВА (ДОМОДЕДОВО),САНКТ-ПЕТЕРБУРГ (ПУЛКОВО),1.0
0,18581596,Аэрофлот,SU,030,25.11.2022 20:25:00,25.11.2022 21:26:13,True,B737,МОСКВА (ШЕРЕМЕТЬЕВО),САНКТ-ПЕТЕРБУРГ (ПУЛКОВО),1.0
0,18589620,Авиакомпания «Уральские авиалинии»,U6,263,25.11.2022 20:35:00,25.11.2022 21:18:00,True,None,МОСКВА (ДОМОДЕДОВО),ЕКАТЕРИНБУРГ (КОЛЬЦОВО),1.0


In [9]:
result = result.sort_values(['Plan_time', 'Fact_time', 'Departure_airport', 'Arrival_airport', 'Aircraft', 'rus_company'])
result.head(5)

,Flight_id,Company_name,Company_code,Flight_number,Plan_time,Fact_time,Completed_flight,Aircraft,Departure_airport,Arrival_airport,rus_company
0,18901464,Аэрофлот,SU,1524,01.01.2023 0:05:00,01.01.2023 0:05:00,False,B737,МОСКВА (ШЕРЕМЕТЬЕВО),ГОРНО-АЛТАЙСК (ГОРНО-АЛТАЙСК),1.0
0,18893383,S7 Airlines,S7,1131,01.01.2023 0:05:00,01.01.2023 0:26:00,True,B737,МОСКВА (ДОМОДЕДОВО),УФА (УФА),1.0
0,18892368,Аэрофлот,SU,276,01.01.2023 0:15:00,01.01.2023 0:23:41,True,B777,МОСКВА (ШЕРЕМЕТЬЕВО),ПХУКЕТ (ПХУКЕТ),1.0
0,18893384,S7 Airlines,S7,2613,01.01.2023 0:15:00,01.01.2023 0:28:00,True,A320,МОСКВА (ДОМОДЕДОВО),ТОМСК (БОГАШЕВО),1.0
0,18893388,S7 Airlines,S7,2609,01.01.2023 0:20:00,01.01.2023 0:24:00,True,None,МОСКВА (ДОМОДЕДОВО),НИЖНЕВАРТОВСК (НИЖНЕВАРТОВСК),1.0


In [10]:
# result.to_csv('MOW_DEPARTURES_20220101_20230301_v4.csv', sep='\t', encoding='utf-8')

In [11]:
result.to_excel('MOW_DEPARTURES_from_20221126.xlsx', encoding='utf-8')